In [8]:
from include.road_process import Road
from include.cool_space import CoolSpace
from include.building import Building
import geopandas as gpd
import rasterio
import matplotlib.pyplot as plt

In [9]:
directory_mac = "/Volumes/T7 Shield/TUD/Synthesis/cool_place/"
directory_win = "G:\\TUD\\Synthesis\\cool_place\\"

landuse_file = directory_win + "ams_landuse_top10NL.shp"
road_file = directory_win + "ams_roads_top10NL.shp"
building_file = directory_win + "ams_buildings_bagplus.shp"
shadow_file = directory_win + "testShadow.tif"

In [ ]:
road = Road(gpd.read_file(road_file))

In [ ]:
building = Building(gpd.read_file(building_file))

In [ ]:
coolSpace = CoolSpace(gpd.read_file(landuse_file))

In [ ]:
shadows = []
shadow_map1 = rasterio.open(shadow_file, crs=coolSpace.data.crs)
print(shadow_map1.crs)
shadows.append(shadow_map1)

In [ ]:
road.create_attribute('typeweg', 'buffer')
road.create_buffer('buffer')
building.create_buffer(4)

In [ ]:
road.data = road.data.set_geometry("buffered")
coolSpace.clip(coolSpace.data, road.data)

building.data = building.data.set_geometry("buffered")
coolSpace.data = coolSpace.data.set_geometry("clipped")
coolSpace.clip(coolSpace.data, building.data)

In [ ]:
coolSpace.data["clipped"].plot()
plt.show()

In [ ]:
coolSpace.calculate_shade(shadows, use_clip=True)

In [ ]:
# 打印 shadeGeom0 列中不是 None 的行
non_none_rows = coolSpace.data[coolSpace.data["shadeGeom0"].notna()]
print(non_none_rows[["shadeGeom0"]]["shadeGeom0"].area.max())
print(non_none_rows[["shadeGeom0"]]["shadeGeom0"].area.min())

In [ ]:
coolSpace.data["shadeGeom0"].plot()
plt.show()

In [ ]:
coolSpace_clipped = gpd.GeoDataFrame(coolSpace.data, geometry='clipped', crs=coolSpace.data.crs)
coolSpace_clipped.drop(columns=["geometry"], inplace=True)
coolSpace_clipped.to_file(directory_win + "test.shp")

coolSpace_origin = gpd.GeoDataFrame(coolSpace.data, geometry='geometry', crs=coolSpace.data.crs)
coolSpace_origin.drop(columns=["clipped"], inplace=True)
coolSpace_origin.to_file(directory_win + "test2.shp")